In [1]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras import layers, Sequential

2024-06-04 23:10:52.820897: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
data_path = '/Users/kun/code/kun1887/TwitPol/data_cleaned.csv'
data = pd.read_csv(data_path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67467 entries, 0 to 67466
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet        67467 non-null  object
 1   y            67467 non-null  bool  
 2   tweet_clean  67120 non-null  object
dtypes: bool(1), object(2)
memory usage: 1.1+ MB


In [5]:
data.sample(10)['tweet_clean']

28220    mean go build bomb go cut health care kid say ...
38702    democratic republican leader senate hold gover...
39878        ann train fine pray comfort family individual
7108     dr james west make black history day creator w...
6604     thank attend recent passport fair unable atten...
6778      livestream question facebook ceo mark zuckerberg
52103    .#taxcuts lead high wage pay parental leave \n...
29272    look forward take live discussion morning achi...
30799    live soon floor talk h.con.re encourage reunio...
14915                             president trump try hide
Name: tweet_clean, dtype: object

In [6]:
X = data.tweet_clean
y = data.y

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [53]:
for x in '1 2 3.':
    print(type(x))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [57]:
my_string = "Hello, World!"
type(my_string)

str

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_tok = tokenizer.texts_to_sequences(X_train)
X_test_tok = tokenizer.texts_to_sequences(X_test)

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
tokenizer.index_word

In [ ]:
np.average([len(x) for x in X_train_tok])

In [ ]:
vocal_size = len(tokenizer.word_index)

In [ ]:
X_train_pad = pad_sequences(X_train_tok, dtype='int', padding='post', maxlen=10)
X_test_pad = pad_sequences(X_test_tok, dtype='int', padding='post')

In [ ]:
print(X_train_pad.shape)
print(X_test_pad.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train_pad[:10]

In [ ]:
def init_model():
    model = Sequential()

    model.add(layers.Embedding(input_dim=vocal_size + 1, output_dim=50))
    model.add(layers.LSTM(5))
    model.add(layers.Dense(12, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = init_model()

history = model.fit(
    X_train_pad,
    y_train,
    epochs=5,
    validation_split=0.3,
    batch_size=128,
    verbose=2
    )

plot_loss_accuracy(history)

In [ ]:
def plot_loss_accuracy(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))

    # --- LOSS ---

    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])

    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')

    ax[0].set_ylim((0,1))

    ax[0].legend(['Train', 'Test'], loc='best')

    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)

    # --- ACCURACY

    ax[1].plot(history.history['accuracy'])
    ax[1].plot(history.history['val_accuracy'])

    ax[1].set_title('Model Accuracy')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_xlabel('Epoch')

    ax[1].legend(['Train', 'Test'], loc='best')

    ax[1].set_ylim((0,1))

    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)

    if title:
        fig.suptitle(title)

In [ ]:
model.summary()

In [ ]:
embedding = model.layers[0].weights[0]._value


In [ ]:
example = ["I vote for Trump"]

tokenized = tokenizer.texts_to_sequences(example)
tokenized

In [ ]:
padded = pad_sequences(tokenized, dtype='int', padding='post', maxlen=10)

In [ ]:
model.predict(padded)

In [ ]:
import lime
from lime.lime_text import LimeTextExplainer

class KerasClassifierWrapper:
    def __init__(self, model, tokenizer, maxlen):
        self.model = model
        self.tokenizer = tokenizer
        self.maxlen = maxlen

    def predict_proba(self, texts):
        sequences = self.tokenizer.texts_to_sequences(texts)
        padded = pad_sequences(sequences, maxlen=self.maxlen, padding='post')
        return np.hstack((1 - self.model.predict(padded), self.model.predict(padded)))

wrapper = KerasClassifierWrapper(model, tokenizer, X_train_pad.shape[1])
ls_X_test = cleaned_texts  # Your test data should be prepared similarly
class_names = {0: 'non-democratic', 1: 'democratic'}

explainer = LimeTextExplainer(class_names=class_names)

idx = np.random.randint(200)
explanation = explainer.explain_instance(ls_X_test[idx], wrapper.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Text: ', ls_X_test[idx])
print('Probability democratic =', wrapper.predict_proba([ls_X_test[idx]]).round(3)[0,1])
print('True class: %s' % class_names[data.iloc[idx]['y']])

# Show the explainability results with highlighted text
print("1 = democratuc class, 0 = non-democratic class")
explanation.show_in_notebook(text=True)
